In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [2]:
training_set = pd.read_csv('data.csv')
training_set = training_set.iloc[:,1:2].values
plt.plot(training_set, label = ' ')
plt.show()

In [3]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

sc = MinMaxScaler()
training_data = sc.fit_transform(training_set)
seq_length = 4
x, y = sliding_windows(training_data, seq_length)

train_size = int(len(y) * 0.5)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

In [4]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [5]:
num_epochs = 4000
learning_rate = 0.01

input_size = 1
hidden_size = 2
num_layers = 1

num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
    loss.backward()
    
    optimizer.step()

In [6]:
lstm.eval()
train_predict = lstm(dataX)

data_predict = train_predict.data.numpy()
dataY_plot = dataY.data.numpy()

data_predict = sc.inverse_transform(data_predict)
dataY_plot = sc.inverse_transform(dataY_plot)

plt.axvline(x=train_size, c='r', linestyle='--')

plt.plot(dataY_plot)
plt.plot(data_predict)
plt.suptitle('Time-Series Prediction')
plt.show()

In [7]:
import time
import sys
import numpy as np
import vrep
from assets import *

In [9]:
class Algorithm:
  def __init__(self, ROBOT, TARGET):
    self.state = States.MOVING

    self.MIN_DETECTION_DIST = 0
    self.MAX_DETECTION_DIST = 1

    self.ROBOT = ROBOT
    self.TARGET = TARGET
    self.SPEED = 1.5
    self.INDENT_DIST = 0.5
    self.ar = []
    self.SLEEP_TIME = 0.2
    self.PI = math.pi
    self.bot_dir = None
    self.bot_pos = None
    self.bot_euler_angles = None

    self.target_pos = None
    self.target_dir = np.zeros(3)

    self.detect = np.zeros(16)

    self._init_client_id()
    self._init_robot_handle()
    self._init_sensor_handles()
    self._init_wheels_handles()
    self._init_target_handle()

    self.obstacle_dist_stab_PID = PIDController(50)
    self.obstacle_follower_PID = PIDController(50)
    self.obstacle_dist_stab_PID.set_coefficients(2, 0, 0.5)
    self.obstacle_follower_PID.set_coefficients(2, 0, 0)

    self.min_dist_to_target = None

    self.print_info()


  def _init_client_id(self):
    vrep.simxFinish(-1)

    self.client_id = vrep.simxStart('127.0.0.1', 19999, True, True, 5000, 5)

    if self.client_id != -1:
      print('')
    else:
      sys.exit('')


  def _init_robot_handle(self):
    error_code, self.bot_handle = vrep.simxGetObjectHandle(self.client_id, self.ROBOT, vrep.simx_opmode_oneshot_wait)


  def _init_sensor_handles(self):
    self.sensor_handles = []

    for x in range(0, 16):
      error_code, sensor_handle = vrep.simxGetObjectHandle(self.client_id, 'Pioneer_p3dx_ultrasonicSensor' + str(x + 1), vrep.simx_opmode_oneshot_wait)
      self.sensor_handles.append(sensor_handle)
      vrep.simxReadProximitySensor(self.client_id, sensor_handle, vrep.simx_opmode_streaming)


  def _init_wheels_handles(self):
    error_code, self.left_motor_handle = vrep.simxGetObjectHandle(self.client_id, 'Pioneer_p3dx_leftMotor', vrep.simx_opmode_oneshot_wait)
    error_code, self.right_motor_handle = vrep.simxGetObjectHandle(self.client_id, 'Pioneer_p3dx_rightMotor', vrep.simx_opmode_oneshot_wait)


  def _init_target_handle(self):
    error_code, self.target_handle = vrep.simxGetObjectHandle(self.client_id, self.TARGET, vrep.simx_opmode_oneshot_wait)


  def _init_values(self):
    # Инициализация значений цели и робота
    error_code, _ = vrep.simxGetObjectPosition(self.client_id, self.target_handle, -1, vrep.simx_opmode_oneshot)
    error_code, _ = vrep.simxGetObjectPosition(self.client_id, self.bot_handle, -1, vrep.simx_opmode_oneshot)
    error_code, _ = vrep.simxGetObjectOrientation(self.client_id, self.bot_handle, -1, vrep.simx_opmode_streaming)


  def read_values(self):
    # Чтение значений позиции цели
    error_code, target_pos = vrep.simxGetObjectPosition(self.client_id, self.target_handle, -1, vrep.simx_opmode_streaming)
    self.target_pos = Vector3(x=target_pos[0], y=target_pos[1], z=0)

    # Чтение значений позиции робота
    error_code, bot_pos = vrep.simxGetObjectPosition(self.client_id, self.bot_handle, -1, vrep.simx_opmode_streaming)
    self.bot_pos = Vector3(x=bot_pos[0], y=bot_pos[1], z=0)

    # Чтение значений углов Эйлера робота
    error_code, bot_euler_angles = vrep.simxGetObjectOrientation(self.client_id, self.bot_handle, -1, vrep.simx_opmode_streaming)
    self.bot_euler_angles = Vector3(x=0, y=0, z=bot_euler_angles[2])


  def stop_move(self):
    # Остановка движения робота
    error_code = vrep.simxSetJointTargetVelocity(self.client_id, self.left_motor_handle,  0, vrep.simx_opmode_streaming)
    error_code = vrep.simxSetJointTargetVelocity(self.client_id, self.right_motor_handle, 0, vrep.simx_opmode_streaming)


  def read_from_sensors(self):
    # Чтение значений с сенсоров
    for i in range(0, 16):
      error_code, detection_state, detected_point, detected_object_handle, detected_surface_normal_vector = vrep.simxReadProximitySensor(self.client_id, self.sensor_handles[i], vrep.simx_opmode_streaming)

      # Значение дистанции до препятствия
      dist = math.sqrt(detected_point[0] ** 2 + detected_point[1] ** 2 + detected_point[2] ** 2)

      # Присвоение значений датчиков: чем больше значение, тем ближе к конкретному датчику препятствие
      if dist < self.MIN_DETECTION_DIST:
        self.detect[i] = self.MIN_DETECTION_DIST
      elif dist > self.MAX_DETECTION_DIST or detection_state == False:
        self.detect[i] = self.MAX_DETECTION_DIST
      else:
        self.detect[i] = self.MAX_DETECTION_DIST - ((dist - self.MAX_DETECTION_DIST) / (self.MIN_DETECTION_DIST - self.MAX_DETECTION_DIST))


  def print_info(self):
    print('Цель: ' + self.TARGET)
    print('Робот: ' + self.ROBOT)
    print('Скорость: ' + str(self.SPEED))


  def tick(self):
    # Задержка программы
    time.sleep(self.SLEEP_TIME)


  def loop(self):
    self._init_values()
    self.read_values()
    i = 0
    # Основной цикл программы
    while Utils.distance_between_points(self.bot_pos, self.target_pos) > 0.4 or Utils.distance_between_points(self.bot_pos, self.target_pos) == 0:
      self.tick()
      # print(Utils.distance_between_points(self.bot_pos, self.target_pos))
      i = i+1
      # Останавливаем движение и читаем данные робота и цели
      self.stop_move()
      self.read_values()

      # Получаем данные с сенсоров
      self.read_from_sensors()

      # Инициализируем новый кватернион
      q_rot = Quaternion()
      # Получаем кватернион из угла разворота вокруг вектора и самого вектора соответственно
      q_rot.set_from_vector(self.bot_euler_angles.z, Vector3(0, 0, 1))
      # Присваиваем вектору робота значения поворота вектора кватернионом
      self.bot_dir = q_rot.rotate(Vector3(1, 0, 0))

      # Проверка текущего состояния и выполнение соответствующей функции
      if self.state == States.MOVING:
        self.action_moving()
      elif self.state == States.ROTATING:
        self.action_rotating()
      elif self.state == States.ROUNDING:
        self.action_rounding()

    print('Цель достигнута')
    self.stop_move()


  def action_moving(self):
    # Если робот оказалася близко к препятствию, объезжаем это препятствие, иначе продолжаем движение до цели
    if (self.detect[4] + self.detect[5])/2 < self.INDENT_DIST:
      # Инициализируем новый кватернион
      q = Quaternion()
      # Получаем кватернион из угла разворота вокруг вектора и самого вектора соответственно
      q.set_from_vector(self.PI/2, Vector3(0, 0, 1))
      # Присваиваем вектору цели значения поворота вектора направления робота кватернионом
      self.target_dir = q.rotate(self.bot_dir)
      # Переключаем состояние
      self.state = States.ROTATING
    else:
      # Значение угла в радианах между векторами направления движения робота и между роботом и целью
      angle = Utils.angle_between_vectors(self.bot_dir, self.target_pos.minus(self.bot_pos))

      # Корректируем движение робота до прямолинейного до цели, если необходимо
      if math.fabs(angle) > self.PI/180:
        vrep.simxSetJointTargetVelocity(self.client_id, self.left_motor_handle,  self.SPEED + angle, vrep.simx_opmode_streaming)
        vrep.simxSetJointTargetVelocity(self.client_id, self.right_motor_handle, self.SPEED - angle, vrep.simx_opmode_streaming)
      else:
        vrep.simxSetJointTargetVelocity(self.client_id, self.left_motor_handle,  self.SPEED, vrep.simx_opmode_streaming)
        vrep.simxSetJointTargetVelocity(self.client_id, self.right_motor_handle, self.SPEED, vrep.simx_opmode_streaming)


  def action_rotating(self):
    # Значение угла в радианах между векторами направления движения робота и цели
    angle = Utils.angle_between_vectors(self.bot_dir, self.target_dir)

    # Поворачиваем робота вокруг своей оси, если необходимо, иначе переключаем состояние
    if math.fabs(angle) > 5 * self.PI/180:
      vrep.simxSetJointTargetVelocity(self.client_id, self.left_motor_handle, angle, vrep.simx_opmode_streaming)
      vrep.simxSetJointTargetVelocity(self.client_id, self.right_motor_handle, -angle, vrep.simx_opmode_streaming)
    else:
      self.state = States.ROUNDING


  def action_rounding(self):
    # Инициализация нового кватерниона
    q = Quaternion()
    # Получаем кватернион из угла разворота вокруг вектора и самого вектора соответственно
    q.set_from_vector(self.PI/2, Vector3(0, 0, 1))
    # Присваиваем перпендикуляру вектора робота значения поворота вектора направления робота кватернионом
    perp_bot_dir = q.rotate(self.bot_dir)

    # Значение угла в радианах между векторами перпендикуляра к роботу и между роботом и целью
    angle = Utils.angle_between_vectors(perp_bot_dir, self.target_pos.minus(self.bot_pos))
    #==============================================================================
    f1 = open('f1.txt', 'a')
    z = str(angle)
    f1.write(z+'\n')
    self.ar.append(angle)
    #print(len(self.ar))
    # Текущая дистанция до цели
    current_dist_to_target1 = Utils.distance_between_points(self.bot_pos, self.target_pos)
    current_dist_to_target = data_predict[len(self.ar)]
    print('predict',current_dist_to_target)
    print('real', current_dist_to_target1)
    #==============================================================================
    f2 = open('f2.txt', 'a')
    m = str(current_dist_to_target)
    f2.write(m+'\n')

    # Если минимальное значение дистанции до цели меньше, чем текущее, то присваиваем минимальному значению текущее
    if self.min_dist_to_target == None or self.min_dist_to_target <= current_dist_to_target:
      self.min_dist_to_target = current_dist_to_target
    # Прекращаем объезд препятствия
    elif math.fabs(angle) < 5.0 / 180 * self.PI:
      self.state = States.MOVING
      return

    # Разница между значениями двух крайних справа датчиков
    delta = self.detect[7] - self.detect[8]

    # Устанавливаем расстояние до препятствия
    obstacle_dist = min([self.detect[7], self.detect[8]]) - self.INDENT_DIST

    obstacle_dist_stab = self.obstacle_dist_stab_PID.output(obstacle_dist)
    obstacle_follower = self.obstacle_follower_PID.output(delta)
  
    vrep.simxSetJointTargetVelocity(self.client_id, self.left_motor_handle, self.SPEED + obstacle_follower + obstacle_dist_stab - (1 - (self.detect[4] + self.detect[5])/2), vrep.simx_opmode_streaming)
    vrep.simxSetJointTargetVelocity(self.client_id, self.right_motor_handle, self.SPEED - obstacle_follower - obstacle_dist_stab + (1 - (self.detect[4] + self.detect[5])/2), vrep.simx_opmode_streaming)


bug = Algorithm(ROBOT='P3DX', TARGET='Destination')

bug.loop()


Цель: Destination
Робот: P3DX
Скорость: 1.5
('predict', array([7.8232365], dtype=float32))
('real', 7.881274065215608)
('predict', array([7.841836], dtype=float32))
('real', 7.881740069866852)
('predict', array([7.861999], dtype=float32))
('real', 7.893409313819327)
('predict', array([7.880374], dtype=float32))
('real', 7.918259426921946)
('predict', array([7.893262], dtype=float32))
('real', 7.932876290553588)
('predict', array([7.9068885], dtype=float32))
('real', 7.9380942882448)
('predict', array([7.9156604], dtype=float32))
('real', 7.930236992950097)
('predict', array([7.920398], dtype=float32))
('real', 7.912954605993055)
('predict', array([7.9200916], dtype=float32))
('real', 7.890270951349736)
('predict', array([7.9131675], dtype=float32))
('real', 7.877056086787368)
('predict', array([7.9003115], dtype=float32))
('real', 7.867034211116444)
('predict', array([7.8925066], dtype=float32))
('real', 7.8598796037792455)
('predict', array([7.882779], dtype=float32))
('real', 7.8522

('predict', array([6.6211066], dtype=float32))
('real', 5.909699388858549)
('predict', array([6.583783], dtype=float32))
('real', 5.926667275337984)
('predict', array([6.554684], dtype=float32))
('real', 5.93882974716049)
('predict', array([6.531669], dtype=float32))
('real', 5.9451702056384494)
('predict', array([6.5076237], dtype=float32))
('real', 5.95432685022367)
('predict', array([6.48076], dtype=float32))
('real', 5.967544739277524)
('predict', array([6.4638715], dtype=float32))
('real', 5.9829786419538395)
('predict', array([6.4391346], dtype=float32))
('real', 6.000796073477492)
('predict', array([6.421606], dtype=float32))
('real', 6.017631312199319)
('predict', array([6.4044104], dtype=float32))
('real', 6.0249620806029975)
('predict', array([6.39381], dtype=float32))
('real', 6.0298504011892105)
('predict', array([6.3800654], dtype=float32))
('real', 6.039269042478439)
('predict', array([6.3741455], dtype=float32))
('real', 6.0534377756835696)
('predict', array([6.364874], 

('predict', array([6.8546295], dtype=float32))
('real', 5.220061124905569)
('predict', array([6.824762], dtype=float32))
('real', 5.179872821234006)
('predict', array([6.79569], dtype=float32))
('real', 5.155756166824539)
('predict', array([6.7632766], dtype=float32))
('real', 5.131192967052808)
('predict', array([6.7375774], dtype=float32))
('real', 5.10709106550815)
('predict', array([6.7007146], dtype=float32))
('real', 5.086861416199351)
('predict', array([6.6678576], dtype=float32))
('real', 5.063555572515072)
('predict', array([6.631933], dtype=float32))
('real', 5.043926848657646)
('predict', array([6.6041255], dtype=float32))
('real', 5.021266986474142)
('predict', array([6.5634794], dtype=float32))
('real', 5.002136722440174)
('predict', array([6.527835], dtype=float32))
('real', 4.977006191987151)
('predict', array([6.488299], dtype=float32))
('real', 4.958530032635457)
('predict', array([6.4583635], dtype=float32))
('real', 4.937118261620985)
('predict', array([6.4139986], d

('predict', array([4.4235344], dtype=float32))
('real', 3.122345564420692)
('predict', array([4.4190826], dtype=float32))
('real', 3.0849516038941003)
('predict', array([2.8156164], dtype=float32))
('real', 3.0462451057226896)
('predict', array([2.5277357], dtype=float32))
('real', 3.01189697874234)
('predict', array([2.678692], dtype=float32))
('real', 2.9643303285454237)
('predict', array([2.7273197], dtype=float32))
('real', 2.9213490626343757)
('predict', array([2.734409], dtype=float32))
('real', 2.8768898094099873)
('predict', array([2.7438192], dtype=float32))
('real', 2.83119581196752)
('predict', array([2.7543318], dtype=float32))
('real', 2.7776586619817425)
('predict', array([2.7651358], dtype=float32))
('real', 2.72262849933158)
('predict', array([2.7741666], dtype=float32))
('real', 2.666216279620767)
('predict', array([2.7829068], dtype=float32))
('real', 2.616510234191113)
('predict', array([2.791016], dtype=float32))
('real', 2.559186189842349)
('predict', array([2.7972

In [146]:
type(testY)

torch.autograd.variable.Variable